# SetUP

In [1]:
setwd("/lustre/user/liclab/jialm/Project/SCOPEC/cleanData/oeMAT/humanBrain_1k_PFCV1_merge/")

In [2]:
suppressPackageStartupMessages({
library(dplyr)
library(tidyverse)
library(data.table)
library(ggplot2)
library(edgeR)
library(RColorBrewer)
library(GenomicRanges)
library(ComplexHeatmap)
library(Signac)
#library(rtracklayer)
#library(igraph)
})

# Functions

In [3]:
getStripesPlotDF <- function(bed3,mat,...){
    L <- lapply(1:dim(bed3)[1],function(idx){
    seq=bed3[idx,1]
    s=bed3[idx,2]
    e=bed3[idx,3]
    RG <- mat %>% filter(seqnames==seq,V1>=s,V2<=e) %>% 
    mutate(s=s,idl=((V1-s)/50000)+1,idr=((V2-s)/50000)+1)
    return(RG)
    })
    
    MAT <- matrix(0,ncol = 201,nrow=201)
    ########
    L.matrix <- lapply(L,function(idx){
    mat <- matrix(0,ncol = 201,nrow=201)
    for(i in 1:dim(idx)[1]){
    id1 <- idx[i,6]
    id2 <- idx[i,7]
    score <- idx[i,3]
    MAT[id1,id2] <- score
    MAT[id2,id1] <- score
    }
    return(MAT)
    })
    
    sumMAT.L <- matrix(0,ncol = 201,nrow=201)
    
    ########
    for (i in 1:length(L.matrix)){
     sumMAT.L <- sumMAT.L + L.matrix[[i]]
    }
    
    ########
    return(sumMAT.L)

}

# group stripes

In [4]:
Ustripes <- read.table("/lustre/user/liclab/jialm/Project/SCOPEC/analysis/stripes/res/allhumanBrain_1k_PFC_V1/all.humanBrain.eNtoRG.PFCV1.1k.stripes_20_50_S+L.edgeR.FDR0.05_log2FC0.5.res.Up.larger1Mb")
Dstripes <- read.table("/lustre/user/liclab/jialm/Project/SCOPEC/analysis/stripes/res/allhumanBrain_1k_PFC_V1/all.humanBrain.eNtoRG.PFCV1.1k.stripes_20_50_S+L.edgeR.FDR0.05_log2FC0.5.res.Down.larger1Mb")
Nstripes <- read.table("/lustre/user/liclab/jialm/Project/SCOPEC/analysis/stripes/res/allhumanBrain_1k_PFC_V1/all.humanBrain.eNtoRG.PFCV1.1k.stripes_20_50_S+L.edgeR.FDR0.05_log2FC0.5.res.Not.larger1Mb")

In [5]:
Inpt <- rbind(Ustripes,
      Dstripes,
      Nstripes) %>% 
mutate(L=V3-V2,R=V6-V5) %>% 
mutate(Group=case_when(L==50000 | L==20000 ~ "Left",
                       R==50000 | R==20000 ~ "Right")) %>%
filter(L==50000 | R==50000) 

In [6]:
UL <- Inpt %>% filter(Group=="Left",V12=="Up") %>% select(V1,V2,V3) %>% 
distinct() %>% mutate(V2=V2-500000,V3=V2+10000000)
DL <- Inpt %>% filter(Group=="Left",V12=="Down") %>% select(V1,V2,V3) %>% 
distinct() %>% mutate(V2=V2-500000,V3=V2+10000000)
NL <- Inpt %>% filter(Group=="Left",V12=="Not") %>% select(V1,V2,V3) %>% 
distinct() %>% mutate(V2=V2-500000,V3=V2+10000000)
UR <- Inpt %>% filter(Group=="Right",V12=="Up") %>% select(V4,V5,V6) %>% set_names(paste0("V",1:3)) %>% 
distinct() %>%mutate(V3=V3+500000,V2=V3-10000000)
DR <- Inpt %>% filter(Group=="Right",V12=="Down") %>% select(V4,V5,V6) %>% set_names(paste0("V",1:3)) %>% 
distinct() %>%mutate(V3=V3+500000,V2=V3-10000000)
NR <- Inpt %>% filter(Group=="Right",V12=="Not") %>% select(V4,V5,V6) %>% set_names(paste0("V",1:3)) %>% 
distinct() %>%mutate(V3=V3+500000,V2=V3-10000000)

In [8]:
UL <- Inpt %>% filter(Group=="Left",V12=="Up") %>% select(V1,V2,V3) %>% 
distinct() %>% mutate(V2=V2-1000000,V3=V2+8000000,G="UL")
DL <- Inpt %>% filter(Group=="Left",V12=="Down") %>% select(V1,V2,V3) %>% 
distinct() %>% mutate(V2=V2-1000000,V3=V2+8000000,G="DL")
NL <- Inpt %>% filter(Group=="Left",V12=="Not") %>% select(V1,V2,V3) %>% 
distinct() %>% mutate(V2=V2-1000000,V3=V2+8000000,G="NL")
UR <- Inpt %>% filter(Group=="Right",V12=="Up") %>% select(V4,V5,V6) %>% set_names(paste0("V",1:3)) %>% 
distinct() %>%mutate(V3=V3+1000000,V2=V3-8000000,G="UR")
DR <- Inpt %>% filter(Group=="Right",V12=="Down") %>% select(V4,V5,V6) %>% set_names(paste0("V",1:3)) %>% 
distinct() %>%mutate(V3=V3+1000000,V2=V3-8000000,G="DR")
NR <- Inpt %>% filter(Group=="Right",V12=="Not") %>% select(V4,V5,V6) %>% set_names(paste0("V",1:3)) %>% 
distinct() %>%mutate(V3=V3+1000000,V2=V3-8000000,G="NR")

In [16]:
all.stripes <- rbind(UL,DL,NL,UR,DR,NR) 

In [21]:
write.table(all.stripes %>% mutate(V2=as.integer(V2),V3=as.integer(V3)),
            "/lustre/user/liclab/jialm/Project/SCOPEC/analysis/stripes/liftOver/humanBrain1K.PFCV1.50kb.larger1Mb.stripes.SAup1Mb-down8Mb.bed",
            col.names = F,row.names = F,sep="\t",quote=F)

In [245]:
dim(UL)
dim(DL)
dim(NL)
dim(UR)
dim(DR)
dim(NR)

[1] 387   3

[1] 55  3

[1] 1589    3

[1] 395   3

[1] 49  3

[1] 1628    3

# LoadData

In [103]:
hRG.50 <- lapply(c(1:22,"X"),function(idx){
    read.table(paste0("./RG.huamnBrain.PFCV1.1k.allValidPairs.hic.oeMAT/RG.huamnBrain.PFCV1.1k.allValidPairs.hic_",idx,"_NONE.50kb.mat")) %>%
    mutate(seqnames=idx) 
})

heN.50 <- lapply(c(1:22,"X"),function(idx){
    read.table(paste0("./eN.huamnBrain.PFCV1.1k.allValidPairs.hic.oeMAT/eN.huamnBrain.PFCV1.1k.allValidPairs.hic_",idx,"_NONE.50kb.mat")) %>%
    mutate(seqnames=idx) 
})


In [104]:
hRG.50.mat <- do.call(rbind,hRG.50) %>% mutate(seqnames=paste0("chr",seqnames))
heN.50.mat <- do.call(rbind,heN.50) %>% mutate(seqnames=paste0("chr",seqnames))

In [105]:
saveRDS(hRG.50.mat,"./humanBrain_PFCV1_1K_RG.50.oe.mat.RDS")
saveRDS(heN.50.mat,"./humanBrain_PFCV1_1K_eN.50.oe.mat.RDS")

In [ ]:
#UL.RG.mat <- getStripesPlotDF(UL,hRG.50.mat)
#UL.eN.mat <- getStripesPlotDF(UL,heN.50.mat)
#DL.RG.mat <- getStripesPlotDF(DL,hRG.50.mat)
#DL.eN.mat <- getStripesPlotDF(DL,heN.50.mat)
#NL.RG.mat <- getStripesPlotDF(NL,hRG.50.mat)
#NL.eN.mat <- getStripesPlotDF(NL,heN.50.mat)

#UR.RG.mat <- getStripesPlotDF(UR,hRG.50.mat)
#UR.eN.mat <- getStripesPlotDF(UR,heN.50.mat)
#DR.RG.mat <- getStripesPlotDF(DR,hRG.50.mat)
#DR.eN.mat <- getStripesPlotDF(DR,heN.50.mat)
#NR.RG.mat <- getStripesPlotDF(NR,hRG.50.mat)
#NR.eN.mat <- getStripesPlotDF(NR,heN.50.mat)

In [ ]:
saveRDS(UL.RG.mat,"./humanBrain_1K_PFCV1_UL.RG.mat.RDS")
saveRDS(UL.eN.mat,"./humanBrain_1K_PFCV1_UL.eN.mat.RDS")
saveRDS(DL.RG.mat,"./humanBrain_1K_PFCV1_DL.RG.mat.RDS")
saveRDS(DL.eN.mat,"./humanBrain_1K_PFCV1_DL.eN.mat.RDS")
saveRDS(NL.RG.mat,"./humanBrain_1K_PFCV1_NL.RG.mat.RDS")
saveRDS(NL.eN.mat,"./humanBrain_1K_PFCV1_NL.eN.mat.RDS")
saveRDS(UR.RG.mat,"./humanBrain_1K_PFCV1_UR.RG.mat.RDS")
saveRDS(UR.eN.mat,"./humanBrain_1K_PFCV1_UR.eN.mat.RDS")
saveRDS(DR.RG.mat,"./humanBrain_1K_PFCV1_DR.RG.mat.RDS")
saveRDS(DR.eN.mat,"./humanBrain_1K_PFCV1_DR.eN.mat.RDS")
saveRDS(NR.RG.mat,"./humanBrain_1K_PFCV1_NR.RG.mat.RDS")
saveRDS(NR.eN.mat,"./humanBrain_1K_PFCV1_NR.eN.mat.RDS")

In [321]:
UL.RG.mat <- readRDS("./humanBrain_1K_PFCV1_UL.RG.mat.RDS")
UL.eN.mat <- readRDS("./humanBrain_1K_PFCV1_UL.eN.mat.RDS")
DL.RG.mat <- readRDS("./humanBrain_1K_PFCV1_DL.RG.mat.RDS")
DL.eN.mat <- readRDS("./humanBrain_1K_PFCV1_DL.eN.mat.RDS")
NL.RG.mat <- readRDS("./humanBrain_1K_PFCV1_NL.RG.mat.RDS")
NL.eN.mat <- readRDS("./humanBrain_1K_PFCV1_NL.eN.mat.RDS")
UR.RG.mat <- readRDS("./humanBrain_1K_PFCV1_UR.RG.mat.RDS")
UR.eN.mat <- readRDS("./humanBrain_1K_PFCV1_UR.eN.mat.RDS")
DR.RG.mat <- readRDS("./humanBrain_1K_PFCV1_DR.RG.mat.RDS")
DR.eN.mat <- readRDS("./humanBrain_1K_PFCV1_DR.eN.mat.RDS")
NR.RG.mat <- readRDS("./humanBrain_1K_PFCV1_NR.RG.mat.RDS")
NR.eN.mat <- readRDS("./humanBrain_1K_PFCV1_NR.eN.mat.RDS")

In [363]:
options(repr.plot.width=3,repr.plot.height=3)
pdf("../../../FINAL_figures/fig2/humanBrain_1K_PFCV1_DL.RG.stripes.heatmap_1to3.pdf",width=3,height=3)
ComplexHeatmap::pheatmap(DL.RG.mat[1:51,1:51]/55,fontsize = 8,legend = F,
                         #color=colours,
                         color=colorRampPalette(c("navy","white","red"))(500),
                         #left_annotation = row_ha,
                         #show_rownames = F,
                         breaks=c(1,2.5),
                         cluster_cols = F,cluster_rows = F,border = FALSE) 
dev.off()

pdf("../../../FINAL_figures/fig2/humanBrain_1K_PFCV1_DL.eN.stripes.heatmap_1to3.pdf",width=3,height=3)
ComplexHeatmap::pheatmap(DL.eN.mat[1:51,1:51]/55,fontsize = 8,legend = F,
                         #color=colours,
                         color=colorRampPalette(c("navy","white","red"))(500),
                         #left_annotation = row_ha,
                         #show_rownames = F,
                         breaks=c(1,2.5),
                         cluster_cols = F,cluster_rows = F,border = FALSE) 
dev.off()

pdf("../../../FINAL_figures/fig2/humanBrain_1K_PFCV1_UL.RG.stripes.heatmap_1to3.pdf",width=3,height=3)
ComplexHeatmap::pheatmap(UL.RG.mat[1:101,1:101]/387,fontsize = 8,legend = F,
                         #color=colours,
                         color=colorRampPalette(c("navy","white","red"))(500),
                         #left_annotation = row_ha,
                         #show_rownames = F,
                         breaks=c(1,2.5),
                         cluster_cols = F,cluster_rows = F,border = FALSE) 
dev.off()

pdf("../../../FINAL_figures/fig2/humanBrain_1K_PFCV1_UL.eN.stripes.heatmap_1to3.pdf",width=3,height=3)
ComplexHeatmap::pheatmap(UL.eN.mat[1:101,1:101]/387,fontsize = 8,legend = F,
                         #color=colours,
                         color=colorRampPalette(c("navy","white","red"))(500),
                         #left_annotation = row_ha,
                         #show_rownames = F,
                         breaks=c(1,2.5),
                         cluster_cols = F,cluster_rows = F,border = FALSE) 
dev.off()

pdf("../../../FINAL_figures/fig2/humanBrain_1K_PFCV1_NL.RG.stripes.heatmap_1to3.pdf",width=3,height=3)
ComplexHeatmap::pheatmap(NL.RG.mat[1:101,1:101]/1589,fontsize = 8,
                         #color=colours,
                         color=colorRampPalette(c("navy","white","red"))(500),
                         #left_annotation = row_ha,
                         #show_rownames = F,
                         breaks=c(1,2.5),
                         cluster_cols = F,cluster_rows = F,border = FALSE) 
dev.off()

pdf("../../../FINAL_figures/fig2/humanBrain_1K_PFCV1_NL.eN.stripes.heatmap_1to3.pdf",width=3,height=3)
ComplexHeatmap::pheatmap(NL.eN.mat[1:101,1:101]/1589,fontsize = 8,legend = F,
                         #color=colours,
                         color=colorRampPalette(c("navy","white","red"))(500),
                         #left_annotation = row_ha,
                         #show_rownames = F,
                         breaks=c(1,2.5),
                         cluster_cols = F,cluster_rows = F,border = FALSE) 
dev.off()



Warning message:
“`breaks` does not have the same length as `color`. The colors are
interpolated from the minimal to the maximal of `breaks`.”


png 
  2

Warning message:
“`breaks` does not have the same length as `color`. The colors are
interpolated from the minimal to the maximal of `breaks`.”


png 
  2

Warning message:
“`breaks` does not have the same length as `color`. The colors are
interpolated from the minimal to the maximal of `breaks`.”


...

png 
  2

Warning message:
“`breaks` does not have the same length as `color`. The colors are
interpolated from the minimal to the maximal of `breaks`.”


png 
  2

Warning message:
“`breaks` does not have the same length as `color`. The colors are
interpolated from the minimal to the maximal of `breaks`.”


png 
  2

In [362]:
pdf("../../../FINAL_figures/fig2/humanBrain_1K_PFCV1_DR.RG.stripes.heatmap_1to3.pdf",width=3,height=3)
ComplexHeatmap::pheatmap(DR.RG.mat[151:201,151:201]/49,fontsize = 8,
                         #color=colours,
                         color=colorRampPalette(c("navy","white","red"))(500),legend = F,
                         #left_annotation = row_ha,
                         #show_rownames = F,
                         breaks=c(1,2.5),
                         cluster_cols = F,cluster_rows = F,border = FALSE) 
dev.off()

pdf("../../../FINAL_figures/fig2/humanBrain_1K_PFCV1_DR.eN.stripes.heatmap_1to3.pdf",width=3,height=3)
ComplexHeatmap::pheatmap(DR.eN.mat[151:201,151:201]/49,fontsize = 8,
                         #color=colours,
                         color=colorRampPalette(c("navy","white","red"))(500),legend = F,
                         #left_annotation = row_ha,
                         #show_rownames = F,
                         breaks=c(1,2.5),
                         cluster_cols = F,cluster_rows = F,border = FALSE) 
dev.off()

pdf("../../../FINAL_figures/fig2/humanBrain_1K_PFCV1_UR.RG.stripes.heatmap_1to3.pdf",width=3,height=3)
ComplexHeatmap::pheatmap(UR.RG.mat[101:201,101:201]/395,fontsize = 8,
                         #color=colours,
                         color=colorRampPalette(c("navy","white","red"))(500),legend = F,
                         #left_annotation = row_ha,
                         #show_rownames = F,
                         breaks=c(1,2.5),
                         cluster_cols = F,cluster_rows = F,border = FALSE) 
dev.off()

pdf("../../../FINAL_figures/fig2/humanBrain_1K_PFCV1_UR.eN.stripes.heatmap_1to3.pdf",width=3,height=3)
ComplexHeatmap::pheatmap(UR.eN.mat[101:201,101:201]/395,fontsize = 8,
                         #color=colours,
                         color=colorRampPalette(c("navy","white","red"))(500),legend = F,
                         #left_annotation = row_ha,
                         #show_rownames = F,
                         breaks=c(1,2.5),
                         cluster_cols = F,cluster_rows = F,border = FALSE) 
dev.off()

pdf("../../../FINAL_figures/fig2/humanBrain_1K_PFCV1_NR.RG.stripes.heatmap_1to3.pdf",width=3,height=3)
ComplexHeatmap::pheatmap(NR.RG.mat[101:201,101:201]/1628,fontsize = 8,
                         #color=colours,
                         color=colorRampPalette(c("navy","white","red"))(500),legend = F,
                         #left_annotation = row_ha,
                         #show_rownames = F,
                         breaks=c(1,2.5),
                         cluster_cols = F,cluster_rows = F,border = FALSE) 
dev.off()

pdf("../../../FINAL_figures/fig2/humanBrain_1K_PFCV1_NR.eN.stripes.heatmap_1to3.pdf",width=3,height=3)
ComplexHeatmap::pheatmap(NR.eN.mat[101:201,101:201]/1628,fontsize = 8,
                         #color=colours,
                         color=colorRampPalette(c("navy","white","red"))(500),legend = F,
                         #left_annotation = row_ha,
                         #show_rownames = F,
                         breaks=c(1,2.5),
                         cluster_cols = F,cluster_rows = F,border = FALSE) 
dev.off()

Warning message:
“`breaks` does not have the same length as `color`. The colors are
interpolated from the minimal to the maximal of `breaks`.”


png 
  2

Warning message:
“`breaks` does not have the same length as `color`. The colors are
interpolated from the minimal to the maximal of `breaks`.”


png 
  2

Warning message:
“`breaks` does not have the same length as `color`. The colors are
interpolated from the minimal to the maximal of `breaks`.”


...

png 
  2

Warning message:
“`breaks` does not have the same length as `color`. The colors are
interpolated from the minimal to the maximal of `breaks`.”


png 
  2

Warning message:
“`breaks` does not have the same length as `color`. The colors are
interpolated from the minimal to the maximal of `breaks`.”


png 
  2